In [1]:
import numpy as np
# import scipy as np

In [25]:
""" 
    General data structures like twists and screws
    @author Wil Louis Rothman
    November 2024 - December 2024
"""

import numpy as np

class Twist:
    """
        General twist as defined in EECS C106A.

        >>> v, omega = [3, 4, 5], [pi, pi/2, 0]
        >>> xi = Twist(v, omega)
        >>> xi
        Twist(v = [3, 4, 5], ω = [pi, pi/2, 0])
        >>> 2 * xi.as_array()
        np.array([6, 8, 10, 2pi, pi, 0])
        >>> 2 * xi.vector # same as previous
        np.array([6, 8, 10, 2pi, pi, 0])

    """
    def __init__(self, linear_velocity, angular_velocity):
        linear_velocity, angular_velocity = list(linear_velocity), list(angular_velocity)

        assert len(linear_velocity) == 3 and len(angular_velocity) == 3, \
            f"linear and angular velocities must be 3D vectors, \
                instead got {linear_velocity} (length {len(linear_velocity)}) \
                    and {angular_velocity} (length {len(angular_velocity)})"
        
        self.vector = np.array(list(linear_velocity) + list(angular_velocity))

        assert np.issubdtype(self.vector.dtype, np.number), \
            "linear and angular velocities must be representable by a \
                numerical value"
        
    def __repr__(self):
        return f"Twist(v = {self.vector[:3]}, ω = {self.vector[3:]})"

    def as_array(self):
        return self.vector
    

class Revolute(Twist):
    """
        Pure rotation as defined in EECS C106A.
        xi = [-ω x q    ω]^T
        where omega := angular velocity
        and       q := some point on rotation axis
    """
    def __init__(self, omega, q):
        super().__init__(-np.cross(omega, q), omega)

class Prismatic(Twist):
    """
        Pure translation as defined in EECS C106A.
        xi = [v     0]^T
        where v := linear velocity
    """
    def __init__(self, v):
        super().__init__(v, [0, 0, 0])

class GeneralScrew(Twist):
    """
        General screw motion as defined in EECS C106A.
        xi = [-ω x q + hω   ω]^T
        where h := is pitch
    """
    def __init__(self, omega, q, h):
        super().__init__(-np.cross(omega, q) + h * omega, omega)

In [26]:
v, omega = [3, 4, 5], [np.pi, np.pi/2, 0]
xi = Twist(v, omega)
xi

Twist(v = [3. 4. 5.], ω = [3.14159265 1.57079633 0.        ])

In [27]:
Revolute([0, np.pi / 2, 0], [1, 0, 0])

Twist(v = [-0.         -0.          1.57079633], ω = [0.         1.57079633 0.        ])

In [28]:
Prismatic([3, 4, 5])

Twist(v = [3 4 5], ω = [0 0 0])

In [29]:
GeneralScrew([0, np.pi / 2, 0], [0, 1, 0], 1)

Twist(v = [0.         1.57079633 0.        ], ω = [0.         1.57079633 0.        ])

Repulsion model

In [30]:
class Pole:
    """
        Represents the pole. We are given two (unequal) points of the pole
    """
    def __init__(self, q_pole1, q_pole2):
        q_pole1, q_pole2 = list(q_pole1), list(q_pole2)
        assert len(q_pole1) == 3 and len(q_pole2) == 3, f"q_pole1 and q_pole2 must be 3D coordinates but instead got q1: {q_pole1} and q2: {q_pole2}"
        assert q_pole1 != q_pole2, f"Cannot define pole line as q1 = q2 = {q_pole1}"
        self.q1, self.q2 = np.array(q_pole1), np.array(q_pole2)

    def dist(self, p):
        """ Outputs distance between the line representing the pole and 3D coordinate vector p """
        v = self.q2 - self.q1
        w = p - self.q1
        return np.linalg.norm(np.cross(w, v)) / np.linalg.norm(v)

In [31]:
pole = Pole([0, 0, 0], [0, 0, 1])
pole.dist([1, 1, 1])

np.float64(1.4142135623730951)

In [32]:
!pip install scipy

You should consider upgrading via the 'C:\Users\Wil\ee106a-final\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [75]:
import numpy as np

class Pole:
    def __init__(self, base, tip):
        self.base = np.array(base)
        self.tip = np.array(tip)
        self.direction = (self.tip - self.base) / np.linalg.norm(self.tip - self.base)

class RepulsionModel:
    def __init__(self, pole, x_ef, k=1):
        self.pole = pole
        self.x_ef = np.array(x_ef)
        self.k = k

    @property
    def repulsion_force(self):
        # Vector from pole base to x_ef
        w = self.x_ef - self.pole.base

        # Closest point on the pole
        projection_length = np.dot(w, self.pole.direction)
        closest_point = self.pole.base + projection_length * self.pole.direction

        # Direction of repulsion
        repulsion_vector = self.x_ef - closest_point
        distance = np.linalg.norm(repulsion_vector)

        # Handle edge case: avoid division by zero
        if distance == 0:
            return np.zeros_like(repulsion_vector)

        # Repulsion force (inverse square law, scaled by k)
        return (repulsion_vector / distance**3) * self.k


In [79]:
pole = Pole([0, 0, 0], [0, 0, 1])  # Pole along the z-axis
x_ef = [3, 0, 0]  # Point on the x-axis
repulsion_model = RepulsionModel(pole, x_ef, k=1)
force = repulsion_model.repulsion_force
print(f"x_ef = {x_ef}, repulsion = {force}")


x_ef = [3, 0, 0], repulsion = [0.11111111 0.         0.        ]


In [91]:
round(1.11, ndigits=1)

1.1

In [103]:
from math import *
def random_triple():
   return np.random.rand(3) * 20 - 10


for _ in range(10):
   x_ef = random_triple()
   pole = Pole([0, 0, 0], [1, 1, 1])  # Pole along the z-axis
   repulsion_model = RepulsionModel(pole, x_ef, k=100)
   force = repulsion_model.repulsion_force
   # print(f"x_ef = {x_ef}, repulsion = {force}")
   x = []
   for component in x_ef:
      x.append(int(component * 10) / 10)
   x = tuple(x)
   print(x)

   f = []
   for component in force:
      f.append(int(component * 1000000) / 1000000)
   f = tuple(f)

   print(f"vector((0,0,0),\t {force})")

(-2.2, 8.6, 3.0)
vector((0,0,0),	 [-1.17411021  1.192849   -0.01873879])
(-7.9, -7.9, 1.0)
vector((0,0,0),	 [-0.75280326 -0.7583975   1.51120076])
(3.1, 3.5, -2.9)
vector((0,0,0),	 [ 1.3881445   1.63036923 -3.01851372])
(-0.2, -7.8, -2.4)
vector((0,0,0),	 [ 1.88641954 -2.50682083  0.62040129])
(7.7, -5.3, -3.6)
vector((0,0,0),	 [ 0.80226318 -0.48340984 -0.31885334])
(-7.3, -9.0, -9.3)
vector((0,0,0),	 [ 36.78632769 -14.93346062 -21.85286707])
(-8.1, -5.3, -2.6)
vector((0,0,0),	 [-4.57786871  0.0144051   4.56346361])
(4.0, 7.9, -6.3)
vector((0,0,0),	 [ 0.18841419  0.53496459 -0.72337878])
(-9.6, 2.4, -9.6)
vector((0,0,0),	 [-0.41946835  0.83463033 -0.41516199])
(-7.3, -4.2, -1.2)
vector((0,0,0),	 [-3.83727929  0.0415325   3.79574679])


In [18]:
repulsion_model.repulsion_force

np.float64(0.002338540701304904)

In [36]:
random_triple()

array([ 2.152555  , -0.1117353 ,  0.93682594])

In [2]:
np.random.random(sample_size := 100) * 2 - 1 

array([ 0.18212493, -0.79847025, -0.56233708, -0.24052621, -0.39596245,
       -0.27561829, -0.29887131,  0.36209808, -0.5393991 , -0.78022411,
       -0.69988856,  0.83886873, -0.02155887,  0.29459429,  0.20700091,
        0.62789234, -0.09285226,  0.14347012,  0.33922324, -0.96767844,
        0.25232249,  0.64496507,  0.56988907,  0.82275249, -0.07067283,
        0.07678389,  0.30725388,  0.64558971, -0.19996274,  0.09133624,
        0.7437071 ,  0.82706078, -0.52128678,  0.56009302, -0.65929406,
        0.2781947 ,  0.62304281,  0.07105722, -0.87210293,  0.97430927,
        0.65532858, -0.54417904, -0.63275631,  0.67268998,  0.34459065,
        0.26137025,  0.02165842,  0.0882294 ,  0.331051  ,  0.65916129,
       -0.26203777, -0.23509587,  0.56194072,  0.98674224,  0.4346377 ,
       -0.93109444, -0.96038088, -0.10383338, -0.81988596, -0.34316155,
       -0.05486885,  0.64613248,  0.90649673,  0.29740545,  0.50503888,
        0.90160143,  0.15844354,  0.85230665,  0.30755766, -0.41